In [208]:

include("../phase1/node.jl")
include("../phase1/edges.jl")
include("../phase1/graph.jl")
include("../phase1/read_stsp.jl")
include("comp_connexes.jl")
include("../phase1/main.jl")
include("queue.jl")
include("heuristics.jl")

SystemError: SystemError: opening file "/Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase3/comp_connexes.jl": No such file or directory

In [209]:
tree_root = TreeNode("1", 0)
child1 = TreeNode("2", 0)
child2 = TreeNode("3", 0)
child3 = TreeNode("4", 0)
child4 = TreeNode("5",0)
child5 = TreeNode("6",0)
child6 = TreeNode("7",0)

tree = Tree("test tree", Vector{TreeNode{Int}}())

first_row = [ child1, child2, child3]
second_row = [child4, child6]
add_node!(tree, tree_root)
for child in first_row
    add_node!(tree, child, parent = tree_root)
end
for child in second_row
    add_node!(tree, child, parent = child3)
end
add_node!(tree, child5, parent = child4)
show(tree)

Node 1 has  rank 0 and 3 children.
It has no parent.
Its children are: 
     2
     3
     4
Node 2 has  rank 0 and 0 children.
Its parent is 1
It has no children.
Node 3 has  rank 0 and 0 children.
Its parent is 1
It has no children.
Node 4 has  rank 0 and 2 children.
Its parent is 1
Its children are: 
     5
     7
Node 5 has  rank 0 and 1 children.
Its parent is 4
Its children are: 
     6
Node 7 has  rank 0 and 0 children.
Its parent is 4
It has no children.
Node 6 has  rank 0 and 0 children.
Its parent is 5
It has no children.


In [210]:
find_root(tree)

TreeNode{Int64}("1", 0, nothing, [2, 3, 4], 0, 1)

In [211]:
children_loc(child4)

1-element Vector{Int64}:
 7

In [212]:
println(tree_root)

TreeNode{Int64}("1", 0, nothing, [2, 3, 4], 0, 1)


In [213]:
n1= TreeNode("1", 0)
n2 = TreeNode("2", 0)
n3 = TreeNode("3", 0)
n4= TreeNode("4", 0)
t1 = Tree("test tree" , Vector{TreeNode{Int}}())
t1 = add_node!(t1, n1)
t1 = add_node!(t1, n2, parent = n1)
t1 = add_node!(t1, n3)
t1 = add_node!(t1, n4, parent = n3)
rank_union!(t1, n1, n3)


TreeNode{Int64}("1", 0, nothing, [2, 3], 1, 1)

In [214]:
show(t1)

Node 1 has  rank 1 and 2 children.
It has no parent.
Its children are: 
     2
     3
Node 2 has  rank 0 and 0 children.
Its parent is 1
It has no children.
Node 3 has  rank 0 and 1 children.
Its parent is 1
Its children are: 
     4
Node 4 has  rank 0 and 0 children.
Its parent is 3
It has no children.


In [215]:
"""crée une liste d'adjacence a partir d'un graphe"""
function adjacency_dict( graph::Graph , priority_queue::PriorityQueue{PriorityItem{TreeNode{Y}}}) where {Y}
    #Dictionaire de adjacence
    adj_dict = Dict()
    #Dictionaire de correspondance entre les noeuds et les items de la file de priorite
    correspondance_dict = Dict()
    #Chaque noeud est une clef du dictionaire de adjacence
    for (node, priority_item) in zip(nodes(graph),priority_queue.items)
        adj_dict[priority_item] = Dict()
        correspondance_dict[node] = priority_item
    end
    #Ajoute les voisins de chaque noeud dans le dictionaire de adjacence
    for edge in edges(graph)
        node1, node2 = nodes(edge)
        prior_item1 = correspondance_dict[node1]
        prior_item2 = correspondance_dict[node2]
        adj_dict[prior_item1][prior_item2] = weight(edge)
        adj_dict[prior_item2][prior_item1] = weight(edge)
    end
    return adj_dict
end

"""Crée une file de priorite des arbres à partir d'un graphe. Le noeud de depart a une priorite de 0 et les autres ont une priorite de Inf"""
function prims_priority_queue(graph::Graph{Y,T}, start_node_name::String) where {Y,T}
    priority_queue = PriorityQueue{PriorityItem{TreeNode{Y}}}()
    tree = Tree(name(graph), Vector{TreeNode{Y}}())
    for node in nodes(graph)
        if name(node) == start_node_name
            blank_node = TreeNode(name(node), data(node))
            priority_item = PriorityItem( 0 , blank_node)
        else
            blank_node = TreeNode(name(node), data(node))
            priority_item = PriorityItem( Inf, blank_node)
        end
        add_node!(tree, blank_node)
        push!(priority_queue, priority_item)
    end
    return priority_queue, tree
end

"""Implementation de l'algorithme de Prim"""
function prims_algorithm(graph::Graph{Y,T}; start_node_name::Any = nothing) where {Y,T}
    #initialisation
    if isnothing(start_node_name)
        start_node_name = name(nodes(graph)[1])
    end
    #initialisation de la file de priorite et du dictionaire d'adjacence
    priority_queue, tree = prims_priority_queue(graph, start_node_name)
    adjacency_list = adjacency_dict(graph, priority_queue)
    #sauvegarde de la racine
    root = poplast!(priority_queue)
    priority_node = root
    #boucle principale
    while !is_empty(priority_queue)
        for  (neighbor, edge_weight) in adjacency_list[priority_node]
            #Met a jour la priorite du voisin si elle est dans le priority_que et elle est plus petite que la priorite actuelle
            for item in priority_queue.items
                if name(data(neighbor)) == name(data(item))
                    if  edge_weight < priority(neighbor)
                        change_parent!(tree, data(neighbor), data(priority_node))
                        change_rank!( data(neighbor), edge_weight)
                        priority!(neighbor, edge_weight)
                    end
                break
                end
            end
        end
        #prend le noeud avec la plus petite priorite
        priority_node = poplast!(priority_queue)         
    end
    return tree, data(root)
end

prims_algorithm

In [216]:
graphe_test = Graph("Test",Node{Vector{Float64}}[],Edge{Int,Vector{Float64}}[])

#Nodes 
nodea = Node("a",[0.])
nodeb = Node("b",[0.])
nodec = Node("c",[0.])
noded = Node("d",[0.])
nodee = Node("e",[0.])
nodef = Node("f",[0.])
nodeg = Node("g",[0.])
nodeh = Node("h",[0.])
nodei = Node("i",[0.])
node_list = [nodea,nodeb,nodec, noded,nodee, nodef, 
                nodeg, nodeh, nodei]

#Edges
edge1 = Edge(nodea,nodeb, 4)
edge2 = Edge(nodea,nodeh, 8)
edge3 = Edge(nodeb, nodeh, 11)
edge4 = Edge(nodeb, nodec, 8)
edge5 = Edge(nodeh, nodei, 7)
edge6 = Edge(nodeh, nodeg, 1)
edge7 = Edge(nodeg, nodei, 6)
edge8 = Edge(nodeg, nodef, 2)
edge9 = Edge(nodec, nodef, 4)
edge10 = Edge(nodei, nodec, 2)
edge11 = Edge(nodef, nodee, 10)
edge12 = Edge(nodee, noded, 9)
edge13 = Edge(nodec,noded, 7)
edge14 = Edge(noded, nodef, 14)
edge_list = [edge1, edge2, edge3, edge4, edge5, edge6,
                edge7, edge8, edge9, edge10, edge11,
                edge12, edge13, edge14]

##Construction du graphe
for i in node_list
    add_node!(graphe_test,i)
end
for i in edge_list
    add_edge!(graphe_test, i)
end


In [217]:
priority_queue, tree = prims_priority_queue(graphe_test, "a")

@time p_tree, root = prims_algorithm(graphe_test, start_node_name = "a")

  0.039057 seconds (28.42 k allocations: 1.916 MiB, 99.61% compilation time)


(Tree{Vector{Float64}}("Test", TreeNode{Vector{Float64}}[TreeNode{Vector{Float64}}("a", [0.0], nothing, [2], 0, 1), TreeNode{Vector{Float64}}("b", [0.0], 1, [3], 4, 2), TreeNode{Vector{Float64}}("c", [0.0], 2, [9, 6, 4], 8, 3), TreeNode{Vector{Float64}}("d", [0.0], 3, [5], 7, 4), TreeNode{Vector{Float64}}("e", [0.0], 4, Int64[], 9, 5), TreeNode{Vector{Float64}}("f", [0.0], 3, [7], 4, 6), TreeNode{Vector{Float64}}("g", [0.0], 6, [8], 2, 7), TreeNode{Vector{Float64}}("h", [0.0], 7, Int64[], 1, 8), TreeNode{Vector{Float64}}("i", [0.0], 3, Int64[], 2, 9)]), TreeNode{Vector{Float64}}("a", [0.0], nothing, [2], 0, 1))

In [218]:
@time p_tree1, root1 = prims_algorithm(graphe_test, start_node_name = "a")

  0.000091 seconds (383 allocations: 25.000 KiB)


(Tree{Vector{Float64}}("Test", TreeNode{Vector{Float64}}[TreeNode{Vector{Float64}}("a", [0.0], nothing, [2], 0, 1), TreeNode{Vector{Float64}}("b", [0.0], 1, [3], 4, 2), TreeNode{Vector{Float64}}("c", [0.0], 2, [9, 4, 6], 8, 3), TreeNode{Vector{Float64}}("d", [0.0], 3, [5], 7, 4), TreeNode{Vector{Float64}}("e", [0.0], 4, Int64[], 9, 5), TreeNode{Vector{Float64}}("f", [0.0], 3, [7], 4, 6), TreeNode{Vector{Float64}}("g", [0.0], 6, [8], 2, 7), TreeNode{Vector{Float64}}("h", [0.0], 7, Int64[], 1, 8), TreeNode{Vector{Float64}}("i", [0.0], 3, Int64[], 2, 9)]), TreeNode{Vector{Float64}}("a", [0.0], nothing, [2], 0, 1))

In [219]:
swiss_42_graph, swiss_42_nodes = graph_from_tsp("../../instances/stsp/swiss42.tsp","graphe1")
println("running Prim's algorithm on swiss42")
@time swiss_42_tree, root2 = prims_algorithm(swiss_42_graph)
swiss_42_graph = tree_to_graph(swiss_42_tree, root2)
sum_of_weights(swiss_42_graph)

Reading of header : ✓
Reading of nodes : ✓
Reading of edges : ✓
running Prim's algorithm on swiss42
  0.023194 seconds (17.06 k allocations: 968.016 KiB, 88.35% compilation time)


┌ Warning: ("Le noeud", "1", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:36
┌ Warning: ("Le noeud", "2", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:42
┌ Warning: ("Le noeud", "33", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:42
┌ Warning: ("Le noeud", "7", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:42
┌ Warning: ("Le noeud", "8", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:42
┌ Warning: ("Le noeud", "5", " n'était pas d

1094.0

In [220]:
show(p_tree)

Node a has  rank 0 and 1 children.
It has no parent.
Its children are: 
     b
Node b has  rank 4 and 1 children.
Its parent is a
Its children are: 
     c
Node c has  rank 8 and 3 children.
Its parent is b
Its children are: 
     i
     f
     d
Node d has  rank 7 and 1 children.
Its parent is c
Its children are: 
     e
Node e has  rank 9 and 0 children.
Its parent is d
It has no children.
Node f has  rank 4 and 1 children.
Its parent is c
Its children are: 
     g
Node g has  rank 2 and 1 children.
Its parent is f
Its children are: 
     h
Node h has  rank 1 and 0 children.
Its parent is g
It has no children.
Node i has  rank 2 and 0 children.
Its parent is c
It has no children.


In [222]:
graphe1, graphe1_nodes = graph_from_tsp("../../instances/stsp/bays29.tsp","graphe1")
println("running Prim's algorithm")
@time bays_29_tree, root3 = prims_algorithm(graphe1)
bays_29_graph = tree_to_graph(bays_29_tree, root3)

sum_of_weights(bays_29_graph)

Reading of header : ✓
Reading of nodes : ✓
Reading of edges : ✓
running Prim's algorithm
  0.001131 seconds (4.42 k allocations: 214.234 KiB)


┌ Warning: ("Le noeud", "1", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:36
┌ Warning: ("Le noeud", "21", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:42
┌ Warning: ("Le noeud", "28", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:42
┌ Warning: ("Le noeud", "2", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:42
┌ Warning: ("Le noeud", "6", " n'était pas dans le graphe. Il a été ajouté.")
└ @ Main /Users/letshopethisworks2/Documents/Phd docs/algo_class/mth6412b-starter-code/projet/phase1/graph.jl:42
┌ Warning: ("Le noeud", "20", " n'était pas

1557.0